In [6]:
import pandas as pd
import numpy as np
from pathlib import Path

df = pd.read_csv("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/datasets/preprocessed_data/final_processed_split_data/train.csv")

np.random.seed(42)

label_map = {0: 'negative', 1: 'positive', 2: 'neutral'}
if df["label"].dtype != "object":
    df["label"] = df["label"].map(label_map)

week_biases = {
    "week1": {"positive": 0.6, "negative": 0.2, "neutral": 0.2},
    "week2": {"positive": 0.2, "negative": 0.6, "neutral": 0.2},
    "week3": {"positive": 0.2, "negative": 0.2, "neutral": 0.6},
}

tweets_per_week = 500  

out_dir = Path("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/datasets/sample_data")
out_dir.mkdir(exist_ok=True)

for week, bias in week_biases.items():
    slice_parts = []
    total_weight = sum(bias.values())

    for label, weight in bias.items():
        n_samples = int((weight / total_weight) * tweets_per_week)

        available = df[df["label"] == label]
        if len(available) < n_samples:
            n_samples = len(available)

        sampled = available.sample(n=n_samples, replace=False, random_state=np.random.randint(10000))
        slice_parts.append(sampled)

    week_df = pd.concat(slice_parts).sample(frac=1, random_state=np.random.randint(10000))

    week_df.to_csv(out_dir / f"{week}.csv", index=False)


    dist = week_df["label"].value_counts(normalize=True).round(2).to_dict()
    print(f" Saved {week}.csv with {len(week_df)} tweets | Distribution: {dist}")


 Saved week1.csv with 500 tweets | Distribution: {'positive': 0.6, 'neutral': 0.2, 'negative': 0.2}
 Saved week2.csv with 500 tweets | Distribution: {'negative': 0.6, 'neutral': 0.2, 'positive': 0.2}
 Saved week3.csv with 500 tweets | Distribution: {'neutral': 0.6, 'positive': 0.2, 'negative': 0.2}


In [7]:
import pandas as pd
from pathlib import Path

data_dir = Path("C:/Users/T-Plug/Desktop/University/Level 300/2nd Sem/316 Social Media Mining/End_Of_Sem_Project/datasets/sample_data")

for file in data_dir.glob("week*.csv"):
    df = pd.read_csv(file)

    if "tweet" in df.columns:
        df = df[["tweet"]]
    else:
        raise ValueError(f"'tweet' column not found in {file}")

    df.to_csv(file, index=False)
    print(f"Cleaned {file.name} (removed labels, kept only tweets)")


Cleaned week1.csv (removed labels, kept only tweets)
Cleaned week2.csv (removed labels, kept only tweets)
Cleaned week3.csv (removed labels, kept only tweets)
